In [169]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article             is real or fake:
    1: Fake news
    0: Real news

In [176]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score, classification_report

In [137]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [138]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

> IMPORTING DATA

In [139]:
news = pd.read_csv("/kaggle/input/fake-news/train.csv")
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [140]:
news.shape

(20800, 5)

In [141]:
news.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [142]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [143]:
news = news.fillna("")

In [144]:
x_train = news.drop(columns='label', axis = 1)
y_train = news['label']

In [145]:
news['content'] = news['author']+" "+news['title']

In [146]:
news.drop(columns=['author','title'], axis=1 )

,id,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...
20795,20795,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [147]:
y_train

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

> STEMMING

In [148]:
port_stem = PorterStemmer()

In [149]:
def stemmimg(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in string.punctuation]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [150]:
news['transformed_content'] = news['content'].apply(stemming)

In [151]:
news['transformed_content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: transformed_content, Length: 20800, dtype: object

In [152]:
x_train = news['transformed_content'].values
x_train

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [153]:
y_train = news['label'].values
y_train

array([1, 0, 1, ..., 0, 1, 1])

In [154]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train)

In [155]:
x_train

<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

>SPLIT DATA

In [156]:
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train,test_size=0.2, random_state = 2)

> LOGISTIC REGRESSION

In [157]:
model = LogisticRegression()

In [158]:
model.fit(x_train, y_train)

LogisticRegression()

In [159]:
x_train_pred = model.predict(x_train)

In [160]:
accuracy_train = accuracy_score(y_train,x_train_pred)
accuracy_train

0.9873798076923077

Predicting accuracy score on test data

In [161]:
x_test_pred = model.predict(x_test)

In [162]:
accuracy_test = accuracy_score(x_test_pred, y_test)
accuracy_test

0.9774038461538461

> MODEL EVALUATION

In [177]:
def model_evaluation(model, x_train,x_test,y_train,y_test):
    train_score = pd.Series({'accuracy_score':accuracy_train, 'precision_score':precision_score(y_train,x_train_pred)})
    test_score = pd.Series({'accuracy_score':accuracy_test, 'precision_score':precision_score(x_test_pred, y_test)})
    score_card = pd.concat([train_score,test_score], axis=1)
    
    score_card.columns = ['Train_data','Test_data']
    
    return score_card

In [178]:
model_evaluation(LogisticRegression(), x_train,x_test,y_train,y_test)

,Train_data,Test_data
accuracy_score,0.987380,0.977404
precision_score,0.980824,0.992954


In [174]:
print("----Test Report----")
classification_report(y_test, x_test_pred)

----Test Report----


'              precision    recall  f1-score   support\n\n           0       0.99      0.96      0.98      2031\n           1       0.96      0.99      0.98      2129\n\n    accuracy                           0.98      4160\n   macro avg       0.98      0.98      0.98      4160\nweighted avg       0.98      0.98      0.98      4160\n'

TESTING OF MAODEL

In [188]:
def predict_news(text):
    pred = model.predict(text)
    print(pred)
    if(pred[0] == 0):
        print("The news is Real")
    else:
        print("The news is Fake")

In [189]:
predict_news(x_test[6])

[0]
The news is Real


In [190]:
y_test[6]

0